In [143]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import uszipcode
from uszipcode import SearchEngine


In [2]:
data_path = '/Users/ameliabaum/Desktop/Amelia/Berkeley Fall 2019/CY PLAN 257/Final Project/data/'
notebooks_path = '/Users/ameliabaum/Desktop/Amelia/Berkeley Fall 2019/CY PLAN 257/Final Project/notebooks/'

## Load in the chts tables 

In [3]:
os.listdir(data_path+'chts_csv_data')

['LookUp_PLACE.csv',
 'Deliv_PER.csv',
 'Deliv_HH.csv',
 'LookUp_LD.csv',
 'Deliv_PLACE.csv',
 'LookUp_PER.csv',
 'Deliv_LD.csv',
 'ASSN_TravelDate.csv',
 'Deliv_VEH.csv',
 'LookUp_Home.csv',
 'Deliv_ACTIVITY.csv']

In [4]:
persons = pd.read_csv(data_path+'chts_csv_data/Deliv_PER.csv')
hh = pd.read_csv(data_path+'chts_csv_data/Deliv_HH.csv')
place = pd.read_csv(data_path+'chts_csv_data/Deliv_PLACE.csv')
ld = pd.read_csv(data_path+'chts_csv_data/Deliv_LD.csv')
activity = pd.read_csv(data_path+'chts_csv_data/Deliv_ACTIVITY.csv')[['SAMPN', 'PERNO', 'PLANO', 'APURP', 'O_APURP', 'STIME', 'ETIME']]


/Users/ameliabaum/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (40,60,93,97,104) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ameliabaum/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (25,27,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ameliabaum/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (22,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ameliabaum/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=

### In Place table: create unique record identifiers by person, place and hh
SAMPN is the unique household identifier, PERNO is the person num within the household. HHPER is a unique id for an individual person and includes information about their household

HHPERPLA is the unique id for a person going to a certain place.

Lat Lon information is redacted, so we only have ZIP code, tract and City for place values. This is likely fine because it would make sense to do out analysis at the census tract level.

In [5]:
place['HHPER'] = place['SAMPN'].map(str) + place['PERNO'].map(str) 

place['HHPERPLA'] = place['HHPER'].map(str) + place['PLANO'].map(str)
place1 = place[['HHPERPLA','MODE',"ARR_HR", "ARR_MIN", "DEP_HR", "DEP_MIN","TRIPDUR", "CITY", 'PNAME', 
                'STATE', 'ZIP',"TRACT"]]
place1.head()

,HHPERPLA,MODE,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,TRIPDUR,CITY,PNAME,STATE,ZIP,TRACT
0,103198511,NaN,3,0,8,40,NaN,VALLEJO,HOME,CA,94591,252202.0
1,103198512,6.0,9,2,12,53,22.0,BENICIA,BENICIA SENIOR CITIZEN CENTER,CA,94510,960500.0
2,103198513,6.0,13,13,2,59,20.0,VALLEJO,HOME,CA,94591,252202.0
3,103198521,NaN,3,0,9,16,NaN,VALLEJO,HOME,CA,94591,252202.0
4,103198522,5.0,9,26,10,43,10.0,VALLEJO,LOMA VISTA ELEMENTARY SCHOOL,CA,94589,950200.0


### In activity table: create unique record identifiers by person, place and hh id
CHTS entries are organized into individual trips by survey participants ("persons"), each of which has a destination ("place") and activity purpose ("purpose"), a start time and end time, and an interpolated dwell time. 

This data manipulation step forms uniquely identifiable records for every indidivual trip in the travel diary for that daySAMPN is the unique household identifier, PERNO is the person num within the household. HHPER is a unique id for an individual person and the household that they belong to.

HHPERPLA is the unique identfier for person-place combinations. Two trips by the same person with different destinations will have 2 unique HHPERPLA identifiers. The last number is the place number, the second to last is the person id within the household, and the remaining are the household id.

TSERIES_NUM is the number assigned to a trip within an activity path for each person-place combination. An example of what this represents might be: say a person makes a roundtrip trip from their home, as shown in the image below. The trips that are part of the tour are each assigned a number, and each are to a different place and with a different activity purpose. For instance, trips 1 and 2 are both part of the "home-to-work" trip and so they have the same destination (place) but they may have different purposes (ie: one might be an errand, drop off, getting coffee, etc.)

<img src="activity_model.png" style="width: 400px;">

#### Activity Purposes (from CHTS data dictionary)

|    APURP    | Purpose | Other info | APURP | Purpose | Other info  |
|--------|--------|--------|--------|--------|--------|
| **1** | PERSONAL ACTIVITIES      | SLEEPING, PERSONAL CARE, LEISURE, CHORES      |   **20**    |ALL OTHER AFTER SCHOOL OR NON-CLASS RELATED ACTIVITIES |LIBRARY, BAND REHEARSAL, CLUBS, ETC |
| **2** | PREPARING MEALS/EATING      | 0      |   **21**    |  CHANGE TYPE OF TRANSPORTATION/TRANSFER  | WALK TO BUS, WALK TO/FROM PARKED CAR |
| **3** | HOSTING VISITORS/ENTERTAINING GUESTS  | 0|   **22**    |  PICKUP/DROP OFF PASSENGER(S) | 0    |
| **4**| EXERCISE/PLAYING SPORTS      | WITH OR WITHOUT EQUIPMENT |   **23**   |DRIVE THROUGH MEALS| 0 |
| **5**| STUDY / SCHOOLWORK      | 0      |   **24**    | DRIVE THROUGH OTHER| ATM, BANK|
| **6**| WORK FOR PAY AT HOME   | TELECOMMUNICATIONS EQUIPMENT     |   **25**    |WORK-RELATED |MEETING, SALES CALL, DELIVERY |
| **7**| USING COMPUTER/CELLPHONE FOR PERSONAL ACTIVITIES| 0 | **26** | SERVICE PRIVATE VEHICLE |GAS, OIL, LUBE, REPAIRS |
| **8**| ALL OTHER HOME ACTIVITIES  | 0      |   **27**    |ROUTINE SHOPPING  |GROCERIES, CLOTHING, CONVENIENCE STORE, HH MAINTENANCE |
| **9**| WORK/JOB DUTIES      | 0      |   **28**    |SHOPPING FOR MAJOR PURCHASES OR SPECIALTY ITEMS |APPLIANCE, ELECTRONICS, NEW VEHICLE, MAJOR HH REPAIRS |
| **10**| TRAINING      | 0      |   **29**    |HOUSEHOLD ERRANDS |BANK, DRY CLEANING, ETC. |
| **11**| MEALS AT WORK       | 0      |   **30**    |PERSONAL BUSINESS|EAT MEAL AT RESTAURANT/DINER|
| **12**| WORK-SPONSORED SOCIAL ACTIVITIES      | HOLIDAY OR BIRTHDAY CELEBRATIONS, ETC      |   **31**    | 0| 0    |
| **13**| NON-WORK RELATED ACTIVITIES     | SOCIAL CLUBS, ETC      |   **32**    |HEALTH CARE |DOCTOR, DENTIST, EYE CARE, CHIROPRACTOR, VETERINARIAN|
    | **14**|  EXERCISE/SPORTS      | WITH OR WITHOUT EQUIPTMENT | **33** |CIVIC/RELIGIOUS ACTIVITIES| 0|
| **15**| VOLUNTEER WORK/ACTIVITIES       | 0|   **34**    |OUTDOOR EXERCISE | 0|
| **16**| ALL OTHER WORK-RELATED ACTIVITIES AT MY WORK | 0 |   **35**    |INDOOR EXERCISE |GYM, YOGA, ETC. |
| **17**| IN SCHOOL/CLASSROOM/LABORATORY       | 0 |   **36**    |ENTERTAINMENT |MOVIES, WATCH SPORTS, ETC |
| **18**| MEALS AT SCHOOL/COLLEGE      | 0 | **37**    |SOCIAL/VISIT FRIENDS/RELATIVES| 0|
| **19**| AFTER SCHOOL SPORTS/PHYSICAL ACTIVITY | 0|   **38**    | OTHER (SPECIFY) | 0 |


#### Mode Choices (from CHTS data dictionary)


| Code | **Mode Choice** |Code | **Mode Choice** |
|--------|--------|--------|--------|
| **1** | Walk | **16** | Express Bus / Commuter Bus (AC Transbay, Golden Gate Transit, etc) | 
| **2** | Bike | **17** | Premium Bus ( Metro Orange / Silver Line ) |
| **3** | Wheelchair / Mobility Scooter | **18** | School Bus  |
| **4** | Other Non-Motorized | **19** | Public Transit Shuttle (DASH, Emery Go Round, etc.) |
| **5** | Auto / Van / Truck Driver | **20** | AirBART / LAX FlyAway |
| **6** | Auto / Van / Truck Passenger| **21** | Dial-a-Ride / Paratransit (Access Services, etc.) |
| **7** | Carpool / Vanpool| **22** | Amtrak Bus |
| **8** | Motorcycle / Scooter / Moped| **23** | Other Bus |
| **9** | Taxi / Hired Car / Limo|  **24** | BART, Metro Red / Purple Line|
| **10** | Rental Car/Vehicle |  **25** | ACE, Amtrak, Caltrain, Coaster, Metrolink|
| **11** | Private shuttle (SuperShuttle, employer, hotel, etc.)|  **26** | Metro Blue / Green / Gold Line, Muni Metro, Sacramento Light Rail, San Diego Sprinter / Trolley / Orange/Blue/Green, VTA Light Rail|
| **12** | Greyhound Bus |  **27** | Street Car / Cable Car |
| **13** | Plane|  **28** | Other Rail    |
| **14** | Other Private Transit|  **29** | Ferry / Boat |
| **15** | Local Bus, Rapid Bus |  

In [6]:
activity['HHPER'] = activity['SAMPN'].map(str) + activity['PERNO'].map(str)
activity['HHPERPLA'] = activity['HHPER'].map(str) + activity['PLANO'].map(str)
activity['TSERIES_NUM'] = activity.groupby('HHPERPLA').cumcount() +1 
activity1 = activity.drop("O_APURP", axis =1, inplace=False)

activity2 = pd.merge(place1, activity1, right_on="HHPERPLA", left_on="HHPERPLA")
activity2.head(10)

,HHPERPLA,MODE,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,TRIPDUR,CITY,PNAME,STATE,ZIP,TRACT,SAMPN,PERNO,PLANO,APURP,STIME,ETIME,HHPER,TSERIES_NUM
0,103198511,NaN,3,0,8,40,NaN,VALLEJO,HOME,CA,94591,252202.0,1031985,1,1,1,03:00,07:00,10319851,1
1,103198511,NaN,3,0,8,40,NaN,VALLEJO,HOME,CA,94591,252202.0,1031985,1,1,2,07:00,08:10,10319851,2
2,103198511,NaN,3,0,8,40,NaN,VALLEJO,HOME,CA,94591,252202.0,1031985,1,1,1,08:10,08:40,10319851,3
3,103198512,6.0,9,2,12,53,22.0,BENICIA,BENICIA SENIOR CITIZEN CENTER,CA,94510,960500.0,1031985,1,2,37,09:02,12:48,10319851,1
4,103198513,6.0,13,13,2,59,20.0,VALLEJO,HOME,CA,94591,252202.0,1031985,1,3,2,13:13,13:38,10319851,1
5,103198513,6.0,13,13,2,59,20.0,VALLEJO,HOME,CA,94591,252202.0,1031985,1,3,7,14:10,14:55,10319851,2
6,103198513,6.0,13,13,2,59,20.0,VALLEJO,HOME,CA,94591,252202.0,1031985,1,3,8,15:08,17:12,10319851,3
7,103198521,NaN,3,0,9,16,NaN,VALLEJO,HOME,CA,94591,252202.0,1031985,2,1,1,03:00,06:30,10319852,1
8,103198521,NaN,3,0,9,16,NaN,VALLEJO,HOME,CA,94591,252202.0,1031985,2,1,2,07:30,07:40,10319852,2
9,103198521,NaN,3,0,9,16,NaN,VALLEJO,HOME,CA,94591,252202.0,1031985,2,1,7,08:10,09:16,10319852,3


#### Merge information from the persons table to get demographic information

In [8]:
persons['HHPER'] = persons['SAMPN'].map(str) + persons['PERNO'].map(str)
persons1 = persons[["HHPER", "RELAT", "GEND", "AGE", "HISP", "RACE1"]]
# list(persons.columns)

In [9]:
activity_people = pd.merge(persons1, activity2, right_on="HHPER", left_on="HHPER")

activity_people.head(100)

print("trip records", len(activity_people))

trip records 604710


#### get by zip

In [142]:
zipc_data = pd.DataFrame()
zipc_data["zip code"]= activity_people["ZIP"].unique()

def is_zip(code):
    search = SearchEngine(simple_zipcode=True)
    zipcode = search.by_zipcode(code)
    if (zipcode.zipcode is None):
        return False
    else:
        return True

    

zipc_data["is_zip"] = zipc_data["zip code"].apply(lambda row: is_zip(row))
zipc_data



,zip code,is_zip
0,94591,True
1,94510,True
2,94589,True
3,92127,True
4,92128,True
5,92108,True
6,92111,True
7,92027,True
8,92025,True
9,95341,True


In [91]:
import re
print(len(zipc_data[~(zipc_data["zip code"].str.len().isin([5,6]))]))
print(len(zipc_data))
def is_zip(code):
    '''checks if a zip code is in a standard format. returns 1 if the zip code is a standard 6 or 6-4 code,
    and 0 if it is anything else'''
    if (code.is_integer):
        return len(code) == 6
    
    
r = re.compile('.*/.*/.*:.*')
if r.match('x/x/xxxx xx:xx') is not None:
   print 'matches'
f = np.array(zipc_data["zip code"].apply(lambda code: print(type(code))))
# type(str(f.iloc[1704][0]))
f.unique()

1844
4300
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'st

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

AttributeError: 'numpy.ndarray' object has no attribute 'unique'

In [73]:
zipc_data["zip code"].dtypes

dtype('O')

In [36]:
activity_people["ZIP"].value_counts()

99999          2055
95060          2005
93514          1977
95023          1855
96150          1802
92243          1710
93306          1676
93940          1636
95076          1618
93446          1553
93309          1543
94558          1466
93401          1446
95531          1396
93308          1395
94025          1371
93257          1341
93555          1323
93304          1316
95340          1285
93277          1263
95370          1248
93312          1240
93420          1230
93906          1227
94110          1224
95616          1213
93901          1207
93291          1205
93230          1179
               ... 
SAINT LOUIS       1
30064             1
92155             1
79916             1
79110             1
92199             1
86047             1
12110             1
32344             1
84648             1
43210             1
94999             1
33019             1
22190             1
02141             1
20006             1
80424             1
20502             1
77031             1


In [76]:


for i in range(1,130):
    print(activity_people["STIME"][i], activity_people["STIME"][i] in early_morn)

07:00 True
08:10 False
09:02 False
13:13 False
14:10 False
15:08 False
03:00 False
07:30 False
08:10 False
09:26 False
10:56 False
12:15 False
12:37 False
03:00 False
08:35 False
12:00 False
12:30 False
15:50 False
16:55 False
17:55 False
19:00 False
19:40 False
03:00 False
08:50 False
09:14 False
10:50 False
11:30 False
11:45 False
12:00 False
12:25 False
12:40 False
12:55 False
13:10 False
15:50 False
16:55 False
17:55 False
19:00 False
20:15 False
20:40 False
21:05 False
21:31 False
03:00 False
11:30 False
11:45 False
12:00 False
15:50 False
03:00 False
08:35 False
15:50 False
03:00 False
08:35 False
15:50 False
03:00 False
07:33 False
07:37 False
09:45 False
10:15 False
10:55 False
11:20 False
11:40 False
12:20 False
16:02 False
16:15 False
16:40 False
17:22 False
17:45 False
18:20 False
20:20 False
03:00 False
09:45 False
10:15 False
10:55 False
11:20 False
11:40 False
12:20 False
18:20 False
20:20 False
03:00 False
07:33 False
16:15 False
17:45 False
18:20 False
20:20 False
03:00

#### Create Some Variables

In [82]:
early_morn = pd.Interval(left='03:00', right='06:59')
morn = pd.Interval(left='07:00', right='11:59')
afternoon = pd.Interval(left='12:00', right='16:59')
eve_night = pd.Interval(left='17:00', right='2:59')
def encode_arrival_TOD(a_time):
    '''takes in the set of trips for one person'''
    num_em = 0
    num_m = 0
    num_aft = 0
    num_evn = 0
    if (a_time in early_morn):
        num_em +=1
    elif (a_time in morn):
        num_m +=1
    elif (a_time in afternoon):
        num_aft +=1
    elif (a_time in eve_night):
        num_evn +=1
    return [num_em, num_m, num_aft, num_evn]

In [89]:
activity_people.groupby("HHPER")["STIME"].mean()
# .apply(encode_arrival_TOD)

DataError: No numeric types to aggregate